In [1]:
import sys
sys.path.append("/home/ahow/MyGitHub/oppai_challenge/utils")

from pyspark.sql import functions as F
from logs import print_args
from default_spark import spark
from spark_delta import merge, table_exists
from spark_delta_transform import unnest_struct, transform_column_names

23/11/16 17:28:14 WARN Utils: Your hostname, spiriel resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface enp3s0)
23/11/16 17:28:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ahow/virtual_envs/ahow/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ahow/.ivy2/cache
The jars for the packages stored in: /home/ahow/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-15adc7c7-306b-493e-8745-da348efcd1d2;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 102ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [2]:
class PostsETL:
    def __init__(self, spark, dt_start, dt_end, p_pk=None, t_pk=None):
        self.spark = spark
        self.dt_start = dt_start  # dummy varibles to be used later in real ETL
        self.dt_end = dt_end  # dummy varibles to be used later in real ETL
        self.p_pk, self.t_pk = self._set_pk(p_pk, t_pk)
        
    @staticmethod
    def _set_pk(p_pk, t_pk):
        if p_pk is None:
            p_pk = ["id_oid"]
        if t_pk is None:
            t_pk = ["id_oid", "translations_language"]
        return p_pk, t_pk
            
    @print_args(print_kwargs=['file_name'])
    def extract(self, file_name: str):
        p_df = spark.read.format('json').options(multiLine=True).load(file_name)
        p_df = p_df.withColumn('updated_at_date', F.to_timestamp(p_df['updated_at.$date']))
        p_df = p_df.filter(f"updated_at_date BETWEEN '{self.dt_start}' AND '{self.dt_end}'").drop('updated_at_date')
        return p_df
    
    def transform(self, df):
        # Remove translations to a separated dataframe.
        t_df = df.select("_id", "translations")
        p_df = df.drop("translations")
        
        # Unnest
        t_df = self._transform_translations(df)
        p_df = transform_column_names(unnest_struct(p_df))
        
        t_df = self._transform_to_timestamp(t_df)
        p_df = self._transform_to_timestamp(p_df)
                
        return p_df, t_df
    
    @staticmethod
    def _transform_to_timestamp(df):
        for c in df.columns:
            if '_date' in c:
                df = df.withColumn(c, F.to_timestamp(df[c]))
        return df
    
    @staticmethod
    def _transform_translations(df):
        df = unnest_struct(df)
        df = transform_column_names(df)
        df = df.select(
            "id_oid", F.explode('translations').alias('translations')
        )
        
        # unnest two levels
        df = unnest_struct(df)
        df = unnest_struct(df)
        df = transform_column_names(df)
        return df
    
    @print_args(print_kwargs=['target_p_tb', 'target_t_tb'])
    def load(self, p_df, t_df, target_p_tb, target_t_tb):
        merge(p_df, target_p_tb, self.p_pk, spark_session=self.spark)
        merge(t_df, target_t_tb, self.t_pk, spark_session=self.spark)

In [3]:
%run ./etl_constants.ipynb

In [4]:
etl = PostsETL(spark, DT_START, DT_END)
df = etl.extract(file_name=f"{BASE_PATH}/posts.json")
p_df, t_df = etl.transform(df)
etl.load(
    p_df, t_df,
    target_p_tb=TARGET_POSTS_BRONZE_TB,
    target_t_tb=TARGET_POSTS_TRANSLATIONS_BRONZE_TB
)

--------------------
file_name: ../tests/posts.json
--------------------
--------------------
target_p_tb: oppai_challenge.patreon_posts_bronze
target_t_tb: oppai_challenge.patreon_posts_translations_bronze


23/11/16 17:28:19 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/11/16 17:28:19 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/11/16 17:28:20 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/11/16 17:28:20 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore ahow@127.0.1.1
23/11/16 17:28:20 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
23/11/16 17:28:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


--------------------


In [5]:
spark.sql(f"SELECT COUNT(1) FROM {TARGET_POSTS_BRONZE_TB}").show()
spark.sql(f"SELECT COUNT(1) FROM {TARGET_POSTS_TRANSLATIONS_BRONZE_TB}").show()

+--------+
|count(1)|
+--------+
|     241|
+--------+

+--------+
|count(1)|
+--------+
|    3615|
+--------+



In [6]:
df = spark.read.format('json').options(multiLine=True).load(f"{BASE_PATH}/posts.json")
df.count()

241

In [7]:
df.select(F.sum(F.size('translations'))).show()

+-----------------------+
|sum(size(translations))|
+-----------------------+
|                   3615|
+-----------------------+

